In [3]:
import gurobipy as gb
from gurobipy import GRB

def min_distance(instance):
    file = open(instance,"r")
    text = file.read().split()
    text = text[1:]

    # Extract information from data
    num_team = int(text[0])
    num_round = num_team - 1

    # Create distance matrix D and set upper bound for distances
    D = []
    M = 0
    for i in range(num_team):
        distance_list = []
        for j in range(num_team):
            distance_list.append(int(text[1:][i*num_team+j]))
            M+=int(text[1:][i*num_team+j])
        D.append(distance_list)


    # Set gurobi MIP model
    opt_model = gb.Model("MIP")
    
    # Create MIP model variables
    # Zijp is binary variable
    Z_vars  ={(i, j, p):opt_model.addVar(vtype=gb.GRB.BINARY,
                            name="Z_{0}_{1}_{2}".format(i, j, p))
             for i in range(num_team) for j in range(num_team) for p in range(num_round)}

    # Xip is binary variable
    X_vars  ={(i, p):opt_model.addVar(vtype=gb.GRB.BINARY,
                            name="X_{0}_{1}".format(i, p))
             for i in range(num_team) for p in range(num_round)}

    # Cip is interger variable
    C_vars  ={(i, p):opt_model.addVar(vtype=gb.GRB.INTEGER,
                            lb=0, 
                            ub=M,
                            name="C_{0}_{1}".format(i, p))
             for i in range(num_team) for p in range(num_round-1)}

    # Cmax is interger variable
    L  = opt_model.addVar(vtype=gb.GRB.INTEGER, name='Cmax')

    # Create constraints

    # constraint 1
    constraints = {(i,j):opt_model.addConstr(
        lhs=gb.quicksum(Z_vars[i,j,p] for p in range(num_round)),
        sense=gb.GRB.EQUAL,
        rhs=1, 
        name="constraint4")
    for i in range(num_team) for j in range(num_team) if i != j}

    # constraint 2
    constraints = {(j,p):opt_model.addConstr(
        lhs=gb.quicksum(Z_vars[i,j,p] for i in range(num_team)),
        sense=gb.GRB.EQUAL,
        rhs=1, 
        name="constraint5")
    for j in range(num_team) for p in range(num_round) if j != i}
    
    # constraint 3
    constraints = {(i,j,p):opt_model.addConstr(
        lhs=Z_vars[i,j,p],
        sense=gb.GRB.EQUAL,
        rhs=Z_vars[j,i,p], 
        name="constraint11")
    for i in range(num_team) for j in range(num_team) for p in range(num_round) if i != j}
    
    # constraint 4
    constraints = {p:opt_model.addConstr(
        lhs=gb.quicksum(X_vars[i,p] for i in range(num_team)),
        sense=gb.GRB.EQUAL,
        rhs=num_team/2, 
        name="constraint6")
        for p in range(num_round)}
      
    #constraint 5
    constraints = {(i,j,p):opt_model.addConstr(
        lhs=Z_vars[i,j,p]+X_vars[i,p]+X_vars[j,p],
        sense=gb.GRB.LESS_EQUAL,
        rhs=2, 
        name="constraint7")
        for i in range(num_team) for j in range(num_team) for p in range(num_round) if i != j}

    #constraint 6
    constraints = {(i,j,p):opt_model.addConstr(
        lhs=X_vars[i,p]+X_vars[j,p],
        sense=gb.GRB.GREATER_EQUAL,
        rhs=Z_vars[i,j,p], 
        name="constraint8")
    for i in range(num_team) for j in range(num_team) for p in range(num_round) if i != j}

    # constraint 7
    constraints = {opt_model.addConstr(
                lhs=C_vars[i,p],
                sense=gb.GRB.GREATER_EQUAL,
                rhs=(Z_vars[i,j,p] + X_vars[i,p] + Z_vars[i,k,p+1] + X_vars[k,p+1] -3)*D[i][k], 
                name="constraint1")
    for i in range(num_team) for j in range(num_team) for k in range(num_team) for p in range(num_round-1)
              if i != j if i != k}

    # constraint 8
    constraints = {opt_model.addConstr(
                lhs=C_vars[i,p],
                sense=gb.GRB.GREATER_EQUAL,
                rhs=(Z_vars[i,k,p] + X_vars[k,p] + Z_vars[i,j,p+1] + X_vars[i,p+1] -3)*D[i][k], 
                name="constraint2")
    for i in range(num_team) for j in range(num_team) for k in range(num_team) for p in range(num_round-1)
              if i != j if i != k}

    # constraint 9
    constraints = {opt_model.addConstr(
                lhs=C_vars[i,p],
                sense=gb.GRB.GREATER_EQUAL,
                rhs=(Z_vars[i,j,p] + X_vars[j,p] + Z_vars[i,k,p+1] + X_vars[k,p+1] -3)*D[j][k], 
                name="constraint3")
    for i in range(num_team) for j in range(num_team) for k in range(num_team) for p in range(num_round-1)
              if i != j if i != k}

    # constraint 11
    constraints = {opt_model.addConstr(
        lhs=L,
        sense=gb.GRB.GREATER_EQUAL,
        rhs=gb.quicksum(C_vars[i,p] for i in range(num_team) for p in range(num_round-1)), 
        name="constraint13")}
    
    opt_model.setObjective(L, GRB.MINIMIZE)
    opt_model.optimize()
    
    # optimal objective value
    L = round(opt_model.ObjVal)
    
    # optimal game schedule
    schedule = {}
    
    for i in range(num_round):
        schedule[i] = []
        for j in range(num_team):
            for k in range(num_team):
                if Z_vars[j, k, i].x == 1:
                    if X_vars[j, i].x == 1:
                        schedule[i].append((j, k))
    
    return L, schedule